# Générons une prédiction valide pour le challenge

Imports nécessaires au notebook

In [6]:
import time
import re
import pandas as pd
import numpy as np
import nltk
import sklearn
import pickle
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import tensorflow_datasets as tfds
import multiprocessing

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

Commencez par définir le nom de votre équipe termes alpha numériques sans espace

In [7]:
TEAM_NAME = "Wasabi"
assert re.match("^\w+$", TEAM_NAME) is not None, "Nom d'équipe invalide"

In [8]:
# Chargement des données d'entrainement
train = pd.read_csv("train.csv.gz")
train.head()

,idp,title,description,price,category
0,63610,4pairs silicone Oreillettes de remplacement du...,4pairs silicone Oreillettes de remplacement du...,4.90,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
1,580661,Dvd Rafaela Legouvello,Une aventure humaine hors du commun...,14.89,DVD - BLU-RAY > DVD > DVD DOCUMENTAIRE
2,90191,"Q2671a (H.309Ac) Toner Laser Hp Bleu (Cyan), ...","Q2671A (H.309AC) TONER LASER HP Bleu (Cyan),...",53.40,INFORMATIQUE > IMPRESSION - SCANNER > TONER - ...
3,1297725,Azalaïs ou La vie courtoise,Maryse Rouy,19.05,LIBRAIRIE > LITTERATURE > ROMANS HISTORIQUES
4,119129,Hamecon Coup Vmc Crystal 9408 Bronze (50 N°8),Hameçon coup Crystal 9408 bronzé VMC. Forme ...,7.59,SPORT > PECHE > HAMEÇON


In [9]:
# Chargement des données à prédire
test = pd.read_csv("test.csv.gz")
test.head()

,idp,title,description,price
0,4873686,Endoscope Sans Fil Wifi 1200 P Caméra D'inspec...,"Descriptions&nbsp;Résolution: 640 * 480, 1280 ...",16.85
1,6320508,Aider l'enfant dyslexique,Bernard Jumel 3e édition,16.90
2,6351042,Modà ̈Le réduit de véhicule de construction Li...,Modà¨le réduit de véhicule de construction Lie...,24.97
3,3853418,Lampenwelt lampadaire Led à intensité variable...,Ce lampadaire séduit par son design esthétique...,167.90
4,6373582,"Caméra vidéo caméscope, gordvec 2,7 Pouces écr...","""Caractéristiques : 2,7 pouces (16 : 9) LCD, l...",98.99


Combien y a t'il de catégories produits distinctes dans le dataset?

In [5]:
len(train.category.value_counts())

600

Ca fait beaucoup! Jusqu'ici vous n'aviez que 2 classes à prédire.

## Baseline : 
On va commencer par la prédiction la plus simple possible, une prédiction constante.

On prédit que toutes les catégories sont égales à la 1ère catégorie du dataset de train

In [10]:
category = train.iloc[0].category
print(f"On va toujours prédire: '{category}'")

On va toujours prédire: 'TV - VIDEO - SON > CASQUE - MICROPHONE - DICTAPHONE > CASQUE - ECOUTEURS'


sklearn a même un modèle tout fait pour se genre de cas d'usage : DummyClassifier

In [11]:
model = DummyClassifier(strategy='constant', constant=category)
model.fit(train["title"], train["category"])
y_pred = model.predict(test["title"])

et voici notre prédiction dans le bon format : 2 colonnes, l'identifiant produit suivi de sa catégorie

In [12]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})
prediction.head()

,idp,category
0,4873686,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
1,6320508,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
2,6351042,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
3,3853418,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
4,6373582,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...


## La fonction de publication de vos prédictions

In [13]:
# predictions est un dataframe qui contient au moins les colonnes "idp" et "category"
# Ca écrit simplement le fichier dans le répertoire d'évaluation
def publish_results(predictions):
    now = int(time.time())
    assert re.match("^\w+$", TEAM_NAME) is not None
    filename = f"/home/cisd-jacq/projet/evaluation/prediction-{TEAM_NAME}-{now}.csv.gz"
    predictions[["idp", "category"]].to_csv(filename, index=False, compression="gzip")
    return filename.split("/")[-1]

Une fois que vous avez spécifié un nom d'équipe vous pouvez soumettre votre 1ère prédiction en exécutant la cellule suivante.

In [14]:
# publish_results(prediction)

## La fonction d'évaluation de l'erreur

Vous pouvez utiliser cette fonction pour estimer sur un sous ensemble du dataset train quelle est la précision de votre modèle.

In [15]:
# Plus ce score est grand moins on est content
def error(real_category, predicted_category):
    # On a trouvé la bonne catégorie
    if real_category == predicted_category:
        return 0
    
    # On extrait les sous catégories
    real_categories = real_category.split(" > ")
    pred_categories = predicted_category.split(" > ")
    # On flag les catégories adultes
    adult_categories = ['ADULTE - EROTIQUE', 'VIN - ALCOOL - LIQUIDES']
    real_is_adult = real_categories[0] in adult_categories
    pred_is_adult = pred_categories[0] in adult_categories
    
    # Attention non symmétrie de l'erreur !
    if real_is_adult and not pred_is_adult:
        error = 10_000
        return error
    
    # On identifie à quel niveau on s'est trompé
    for real, pred, error in zip(real_categories, pred_categories, [100, 10, 1]):
        if real != pred:
            return error
    raise ValueError("Catégories différentes, mais aucune différence trouvée")

# Une prédiction moins dummy

Le dataset est bien plus volumineux que les datasets utilisés en TP jusqu'à aujourd'hui.

Si vous vous lancez tête baissée à générer une prédiction vous allez attendre longtemps d'obtenir vos résultats. Il y a 2 raisons à cela : 
- Le nombre de lignes de l'ensemble train est très important
- Le nombre de classes à prédire est aussi très grand. Certains classifier multi-classe génèrent des classifier binaires de type One vs Rest. Si vous avez 600 classes vous allez apprendre 600 classifiers différents

Simplifions donc ces 2 problèmes.

## 1. Travaillons sur 10% du dataset de train

Commencons par itérer rapidement sur le jeu de données. Une fois qu'on aurra un modèle qui nous convient on pourra travailler sur plus de volumétrie.

Générez train_subset, un sample de train faisant 10% de sa taille

In [16]:
train_subset_size = int(0.1*train.shape[0])
chosen_idx = np.random.choice(train_subset_size, replace=False, size=train_subset_size)
train_subset = train.iloc[chosen_idx]

## 2. Limitons les classes

600 classes c'est beaucoup trop. 

On a vu que les catégories contiennent une hiérarchie "catégorie 1 > catégorie 2 > catégorie 3", commencons par travailler uniquement sur les catégories 1.

Générez la colonne "category_1" dans train_subset à partir de la colonne "category" qui contient uniquement la catégorie 1 du produit

In [17]:
def first_category(full_category):
    return full_category.split(">")[0].strip()
if 'category_1' in train_subset.columns:
    train_subset = train_subset.drop(columns=["category_1"])
train_subset.insert(train_subset.shape[1], "category_1", list(map(first_category, train_subset["category"])))

In [18]:
categories_freq = {}
for el in train_subset.iterrows():
    cat1 = el[1]["category_1"]
    if cat1 not in categories_freq:
        categories_freq[cat1] = {"max_freq" : 0, "max_cat" : el[1]["category"]}
    
    if el[1]["category"] not in categories_freq[cat1]:
        categories_freq[cat1][el[1]["category"]] = 1
    else :
        categories_freq[cat1][el[1]["category"]] += 1
        if categories_freq[cat1]["max_freq"] < categories_freq[cat1][el[1]["category"]]:
            categories_freq[cat1]["max_freq"] += 1 
            categories_freq[cat1]["max_cat"] = el[1]["category"]


Combien de "category_1" distinctes vous avez?

In [19]:
nbcat1 = len(np.unique(train_subset["category_1"]))

C'est bien plus raisonnable pour commencer.

## 3. Oh oh il y a pleins de features.

Il y a pleins de features différentes.

- 2 champs de texte title et description 
- 1 champ float : price.

On va se contenter de travailler uniquement avec "title" pour commencer.

Générez les features : X_train

In [20]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

def process_text(line):
    tokens = nltk.word_tokenize(line)
    new_tokens = []
    pattern = '\w'
    for token in tokens:
        if len(token) > 1 and token not in stop_words and re.match(pattern, token):
            new_tokens.append(token.lower())
    return new_tokens


[nltk_data] Downloading package stopwords to /home/cisd-
[nltk_data]     calluau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
if 'title_tokens' in train_subset.columns:
    train_subset = train_subset.drop(columns=["title_tokens"])
    
train_subset.insert(train_subset.shape[1], "title_tokens", np.array([process_text(x) for x in train_subset["title"].to_numpy()]))

In [22]:
def assign_id_to_words(data, field):
    words_to_id = {}
    nextid = 0
    for row in data[field]:
        for word in row:
            if word not in words_to_id:
                words_to_id[word] = nextid
                nextid += 1
    return words_to_id

In [ ]:
if 'title_tokens' in test:
    test = test.drop(columns=["title_tokens"])
    
nproc = 20
manager = multiprocessing.Manager()
title_tokens = manager.list()
titles = test["title"].to_numpy()

jobs = []
def vprocess_text(start, end):
    global title_tokens
    global titles
    for i in range(start, end):
        title_tokens.append(process_text(titles[i]))

for pid in range(nproc):
    job = multiprocessing.Process(target=vprocess_text, args=[int(pid*len(titles)/nproc), len(titles) if pid+1==nproc else int((pid+1)*len(titles)/nproc)])
    job.start()
    jobs.append(job)
    
for job in jobs:
    job.join()

del titles

test.insert(test.shape[1], "title_tokens", np.array(title_tokens))

In [44]:
# if 'title_tokens' in test:
#     test = test.drop(columns=["title_tokens"])
# test.insert(test.shape[1], "title_tokens", np.array([process_text(x) for x in test["title"].to_numpy()]))

In [ ]:
words_to_id = assign_id_to_words(train_subset, "title_tokens")

In [ ]:
categories_to_id = {}
nextid = 0
for cat in train_subset["category_1"]:
    if cat not in categories_to_id:
         categories_to_id[cat] = nextid
         nextid += 1

In [ ]:
id_to_category = {v: k for k, v in categories_to_id.items()}

In [ ]:
def bag_of_words_to_vector(bag_of_words, words_to_id):
    features = np.zeros(len(words_to_id))
    for word in bag_of_words:
        if word in words_to_id:
            features[words_to_id[word]] = 1
    return features

In [ ]:
manager = multiprocessing.Manager()

X_train = manager.list()
y_train = manager.list()

def build_X_train():
    global X_train
    for row in train_subset.iterrows():
        X_train.append(bag_of_words_to_vector(row[1]["title_tokens"], words_to_id))
    X_train = np.array(X_train)
    
def build_y_train():
    global y_train
    for row in train_subset.iterrows():
        y_train.append(categories_to_id[row[1]["category_1"]])
    y_train = np.array(y_train)

p_X_train = multiprocessing.Process(target=build_X_train)
p_y_train = multiprocessing.Process(target=build_y_train)
p_X_train.start()
p_y_train.start()
p_X_train.join()
p_y_train.join()

X_train = np.array(X_train)
y_train = np.array(y_train)

features_dim = len(X_train[0])

In [22]:
X_train

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [26]:
y_train

array([ 0,  1,  2, ..., 10,  9,  9])

## 4. Calculez un modèle avec ces simplifications

Cf TP1 et TP2 : Entrainez une régression Logistique

In [21]:
model = sklearn.linear_model.LogisticRegression()

del train
del train_subset
del stop_words

model.fit(X_train, y_train)

/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
filename = "logistic.model"

In [22]:
pickle.dump(model, open(filename, 'wb'))

In [16]:
model = pickle.load(open(filename, 'rb'))

## 5. Calculez votre prédiction

Appliquez votre modèle sur les données test

In [17]:
confusion_matrix = sklearn.metrics.confusion_matrix(y_train, model.predict(X_train))

NameError: name 'y_train' is not defined

In [26]:
# sklearn.model_selection.cross_validate(model, X_train, y_train, cv=5)

In [27]:
del X_train
del y_train

In [28]:
del confusion_matrix

In [ ]:
confusion_matrix

## 6. Soumettez votre prédiction.

*Pas si vite* : Vous ne prédisez que la catégorie 1. Le script d'évaluation attend une catégorie complète...

C'est simple, pour chaque catégorie 1, choisissez la catégorie de votre choix qui commence par cette "categorie 1".

Modifiez votre prédiction, y_pred, en conséquence.

Soumettez votre prédiction :

In [ ]:
test

In [ ]:
y_pred = []
test_titles = test["title_tokens"].to_numpy()

batch_size = 80000
i = 0
while i < test.shape[0]:
    batch = np.array([bag_of_words_to_vector(title_tokens, words_to_id) for title_tokens in test_titles[i:min(i+batch_size, test.shape[0])]])
    batch_pred = model.predict(batch)
    y_pred = y_pred + list(map(lambda cat_id: categories_freq[id_to_category[cat_id]]["max_cat"], batch_pred))
    i += batch_size
y_pred = np.array(y_pred)

In [ ]:
X_test

In [34]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})

In [35]:
publish_results(prediction)

'prediction-Wasabi-1578867760.csv.gz'

Vous pouvez regardez vos scores en exécutant le notebook [Leaderboard.ipynb](Leaderboard.ipynb). Les données sont mises à jour toutes les 30min.

# Sauvegardez votre modèle

Pour ne pas devoir recommencer à chaque fois tout ce dur labeur, et cette longue attente, vous pouvez sauvegarder votre modèle et votre vectorizer.

La prochaine fois vous n'aurez qu'à les recharger pour faire directement vos prédictions (c'est ce qui est attendu pour la soutenance, sinon le timing sera trop serré) 

La documentation : https://scikit-learn.org/stable/modules/model_persistence.html

# Maintenant à vous de jouer

Vous pouvez commencer par travailler sur plus de volumétrie que 5% du dataset. Mais maintenant le challenge commence.

Si vous ne savez pas par où commencer suivez le déroulement des 2 premiers TPs, en prenant garde à la volumétrie. Ils sont disponibles dans le répertoire ~cisd-jacq/TP/

Contrairement aux TPs vous n'avez pas d'information sur les données de test. (Mis à part le score calculé toute les 30min).

Pour évaluer votre modèle et l'améliorer vous pouvez utiliser les données de train pour crééer un ensemble d'entrainement et un ensemble de validation. 

Vous pourrez alors évaluer plus rapidement vos modèles et identifier quelles sont les catégories sur lesquelles vous devez vous améliorer.

Il n'y a pas que la Régression Logistique dans la vie, essayez d'autres modèles. Je vous ai fait travailler avec sklearn, mais il existe aussi d'autres librairies.

Pour paralléliser vos calculs :
- multiprocessing : https://docs.python.org/3/library/multiprocessing.html
- dask : https://dask.org/ + https://distributed.dask.org/en/latest/ 

In [ ]:
def corpus_generator():
    for row in train.iterrows():
        for word in row[1]["title"]:
            yield word

In [24]:
vocab_filename = "train.vocab"

In [ ]:
encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(corpus_generator(), target_vocab_size=2**15)
encoder.save_to_file(vocab_filename)

In [25]:
encoder = tfds.features.text.SubwordTextEncoder.load_from_file(vocab_filename)

In [26]:
X_train = []
dim = 0

for row in train_subset.iterrows():
    encoded = encoder.encode(row[1]["title"])
    dim = max(dim, len(encoded))
    X_train.append(encoded)

X_train = [x+[0.0 for i in range(dim-len(x)) ] for x in X_train]
X_train = np.array(X_train)

X_train.shape

(30000, 146)

(30000, 146)

In [ ]:
y_train

In [27]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
y_pred = []
test_titles = test["title"].to_numpy()

batch_size = 80000
i = 0
while i < test.shape[0]:
    batch = []
    for title in test_titles[i:min(i+batch_size, test.shape[0])]:
        vec = encoder.encode(title)
        if len(vec) < dim:
            vec = vec + [0.0 for i in range(dim-len(vec))]
        elif len(vec) > dim:
            vec = vec[0:dim]
        if len(vec) != 146:
            print(len(vec))
        batch.append(vec)
    batch = np.array(batch)
    print(batch.shape)
    batch_pred = model.predict(batch)
    y_pred = y_pred + list(map(lambda cat_id: categories_freq[id_to_category[cat_id]]["max_cat"], batch_pred))
    i += batch_size
y_pred = np.array(y_pred)

(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(80000, 146)
(40000, 146)


In [ ]:
X_test[0].shape

In [ ]:
X_test.shape

In [53]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})
publish_results(prediction)

'prediction-Wasabi-1578908336.csv.gz'

In [66]:
def build_model(embedding_dim=64):
    model = tensorflow.keras.Sequential([
        # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
        tensorflow.keras.layers.Embedding(encoder.vocab_size, embedding_dim),
        tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(embedding_dim)),
        # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        # use ReLU in place of tanh function since they are very good alternatives of each other.
        tensorflow.keras.layers.Dense(embedding_dim, activation='relu'),
        # Add a Dense layer with 6 units and softmax activation.
        # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
        tensorflow.keras.layers.Dense(len(np.unique(train_subset["category_1"])), activation='softmax')
    ])
    return model

model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 64)          28416     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 30)                1950      
Total params: 104,670
Trainable params: 104,670
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, verbose=2)

Train on 30000 samples
Epoch 1/10


In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()